This is a notebook to generate all the prompts and seeds in the prompts google sheet

In [1]:
import os
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive')

code_folder = '/content/gdrive/MyDrive/AI Music Visuals Share/New Codes'
fp = os.path.join(code_folder, 'prompts_excel.xlsx')

df_prompt = pd.read_excel(fp, 'prompts', index_col=0).dropna(how='all')
df_prompt

Mounted at /content/gdrive


,prompt,seeds,guidance_scale
name,,,
succulent,"a giant succulent in the desert in the sand, t...",5842234013956479,5.0
reflected galaxy,underground lake with a galaxy reflected in th...,5883559517894420,50.0
red cave1,"descending deep into a cave, red lava on the w...",4866412378884770 4940561922459094,7.5
alien dunes,rolling dunes on an alien planet at night vivi...,5453396741977211 6583682090290432 648251554957...,7.5
cornu,"standing wave pattern, rainbow, wishing well",5842234013956479,50.0
alien mountain,"inspired by David A. Hardy, digital landscape ...",4092123727786949 7285707432238947,7.5
overgrown fantasy,a foggy view of a bridge in a fantasy overgrow...,5618257995059877 4638125635335835 823596687782...,7.5
falling in cave,"descending deep into a cave, red lava on the w...",5451964648290255,7.5
vine cave,a foggy view in a fantasy overgrown vines worl...,7078380737840840,7.5


In [2]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy accelerate

In [3]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                               torch_dtype=torch.float16,
                                               safety_checker=None
                                               )  


pipe = pipe.to("cuda")



/usr/local/lib/python3.9/dist-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


# Iterate through prompts and seeds, outputting an image for both

In [4]:
device = "cuda"
generator = torch.Generator(device=device)
width = 512
height = 512
skip_existing = True

output_folder = os.path.join(code_folder, 'output_prompt_seed')

if not os.path.exists(output_folder): os.makedirs(output_folder)

for name, row in df_prompt.iterrows():
  seeds = row['seeds'].split(' ')
  prompt = row['prompt']
  guidance_scale = float(row['guidance_scale'])

  for seed in seeds:
    output_fn = "{}_{}.png".format(name, seed)

    if os.path.exists(os.path.join(output_folder, output_fn)):
      if skip_existing:
        print("{} already exists, skipping".format(output_fn))
        continue

    generator.manual_seed(int(seed))

    latent = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator,
        device = device
    )

    with torch.autocast(device):
      images = pipe(
          prompt,
          guidance_scale=guidance_scale,
          latents = latent,
      )

    output_image = images.images[0]

    output_image.save(os.path.join(output_folder, output_fn))

succulent_5842234013956479.png already exists, skipping
reflected galaxy_5883559517894420.png already exists, skipping
red cave1_4866412378884770.png already exists, skipping
red cave1_4940561922459094.png already exists, skipping
alien dunes_5453396741977211.png already exists, skipping
alien dunes_6583682090290432.png already exists, skipping
alien dunes_6482515549578068.png already exists, skipping
cornu_5842234013956479.png already exists, skipping
alien mountain_4092123727786949.png already exists, skipping
alien mountain_7285707432238947.png already exists, skipping
overgrown fantasy_5618257995059877.png already exists, skipping
overgrown fantasy_4638125635335835.png already exists, skipping
overgrown fantasy_8235966877828325.png already exists, skipping
falling in cave_5451964648290255.png already exists, skipping
vine cave_7078380737840840.png already exists, skipping
fractal vines_2922127385296522.png already exists, skipping
fractal vines_7102762937299441.png already exists, 

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]